In [1]:
# Install required python packages

# For Linux and Mac
#!pip install -U "qiskit[visualization]", qiskit_ibm_runtime

#!pip install networkx

#!pip install docplex

# %pip install pyyaml

# %pip install python-dotenv

In [2]:
# All imports
import sys
import os

sys.path.append(os.path.abspath(".."))

import rustworkx as rx
from rustworkx.visualization import mpl_draw as draw_graph
import numpy as np
import networkx as nx
import heapq
import json

from qiskit import QuantumCircuit, ClassicalRegister
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.providers.basic_provider import BasicProvider
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_aer import AerSimulator
from scipy.optimize import minimize
from qiskit.circuit.parametervector import ParameterVector
from qiskit.circuit.library import QuadraticForm
from qiskit.circuit.library import qaoa_ansatz
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke

import matplotlib.pyplot as plt
import matplotlib
from scipy.optimize import minimize

from docplex.mp.model import Model

from qiskit_ibm_runtime import SamplerV2 as Sampler

from typing import Sequence

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

import time

import pickle
import seaborn as sns
from qiskit.visualization import plot_histogram

from src.quantum_algorithms.qumda_circuit import QUMDA
from src.quantum_algorithms.qaoa_circuit import QAOA

from dotenv import load_dotenv
from src.utils.config import *
from src.aux import *
from src.utils.paths import *




In [3]:
## Save an IBM Quantum account and set it as your default account.
load_dotenv()
QiskitRuntimeService.save_account(

    token= os.getenv("API_TOKEN"),
    instance=os.getenv("CRN"),
    set_as_default=True,
    # Use `overwrite=True` if you're updating your token.
    overwrite=True,
)


# Load saved credentials
service = QiskitRuntimeService()

In [4]:
def cost_function_qumda(params, ansatz, hamiltonian, estimator):

  # transform the observable defined on virtual qubits to
  # an observable defined on all physical qubits

  global pv, pv_p

  ansatz = qumda.update_distribution_qc(ansatz, pv)

  pub_s = (ansatz, params)
  job_s = sampler.run([(ansatz, params)])
  results_s = job_s.result()[0]
  samp_dist = results_s.data.meas.get_counts()
  shots_array = results_s.data.meas.get_bitstrings()

  pv_p = pv
  pv = UpdateProbabilityVectorCostFunctionAlpha(samp_dist, hamiltonian, num_qubits, 0.1)

  #f_energy
  job_e = estimator.run([(ansatz, hamiltonian, params)])
  results_e = job_e.result()[0]
  cost = results_e.data.evs

  objective_func_vals.append(cost)
  counts.append(samp_dist)
  shots_list.append(shots_array)


  return cost

In [5]:
def cost_function_qumda_w(params, ansatz, hamiltonian, estimator):

  # transform the observable defined on virtual qubits to
  # an observable defined on all physical qubits

  global pv, pv_p

  ansatz = qumda.update_distribution_qc(ansatz, pv)

  pub_s = (ansatz, params)
  job_s = sampler.run([(ansatz, params)])
  results_s = job_s.result()[0]
  samp_dist = results_s.data.meas.get_counts()
  shots_array = results_s.data.meas.get_bitstrings()

  pv_p = pv
  pv = UpdateProbabilityVectorCostFunctionWeighted(samp_dist, hamiltonian, num_qubits, alpha=0.1, epsilon=1e-8)

  #f_energy
  job_e = estimator.run([(ansatz, hamiltonian, params)])
  results_e = job_e.result()[0]
  cost = results_e.data.evs

  objective_func_vals.append(cost)
  counts.append(samp_dist)
  shots_list.append(shots_array)


  return cost

In [6]:
def cost_function_qaoa_estimator(params, ansatz, hamiltonian, estimator):

  # transform the observable defined on virtual qubits to
  # an observable defined on all physical qubits


  #f_opt
  pub_s = (ansatz, params)
  job_s = sampler.run([(ansatz, params)])
  results_s = job_s.result()[0]
  samp_dist = results_s.data.meas.get_counts()
  shots_array = results_s.data.meas.get_bitstrings()


  #f_energy
  job_e = estimator.run([(ansatz, hamiltonian, params)])
  results_e = job_e.result()[0]
  f_energy = results_e.data.evs

  objective_func_vals.append(f_energy)
  counts.append(samp_dist)
  shots_list.append(shots_array)

  return f_energy

# Parameters

In [12]:
config = load_config()
save_dir = ''
circuit_type = 1
circuit = None
alpha_str = ''

qaoa_type_names = ["Vanilla", "Compact", "QUMDA", "QUMDA-W"]
qaoa_type_names = ["QUMDA-W"]
shots_amount = [1024]

classical_optimizer = "minimize"
depth = [1, 2, 3]
depth = [3]
initial_gamma = 0.5
initial_beta = 0.5
initial_theta = 0.5
alpha = 0.1
hardware = True
noise = False
shots = shots_amount[0]
ITERATION_MAX = 35
repeat = 1

loaded_data = read_pickle_files_from_directory(DATA_DIR)


graphs_str = get_graphs_name(loaded_data)[:1]
print(graphs_str)


save_dir = RESULTS_DIR


['10_graph_25_complete']


In [8]:
if hardware:
  chosen_backend = service.least_busy(simulator=False, operational=True)

else:
  if noise:
    chosen_backend = FakeSherbrooke()
    chosen_backend.refresh(service)
  else:
    chosen_backend = AerSimulator()

pm = generate_preset_pass_manager(optimization_level = 3, backend=chosen_backend)

In [13]:
for graph_name in graphs_str:

  graph_sample = loaded_data[graph_name]
  num_qubits = number_qubits(graph_name)
  num_qubits_c = num_qubits - 1

  inital_pv = [0.5] * num_qubits


  Q = get_qubo_matrix_rustworkx(graph_sample)
  max_cut_graph = build_max_cut_paulis(graph_sample)
  cost_hamiltonian = SparsePauliOp.from_list(max_cut_graph)

  Q_reduced, const_offset, index_map = reduce_qubo(Q, fixed_vars=[0])
  J_r, b_r, const_r = qubo_to_ising(Q_reduced)
  graph_sample_r = qubo_to_rustworkx_graph_a(Q_reduced)
  max_cut_graph_r = build_max_cut_paulis(graph_sample_r)
  pauli_terms = max_cut_graph_r.copy()
  linear_terms = []
  for i, coeff in enumerate(b_r):
      if not np.isclose(coeff, 0.0):
          pauli = ['I'] * len(b_r)
          pauli[i] = 'Z'
          linear_terms.append((''.join(pauli), 1.0))

  pauli_terms.extend(linear_terms)
  cost_hamiltonian_r = SparsePauliOp.from_list(pauli_terms)


  for p in depth:

    init_params = p * [initial_gamma] + p * [initial_beta]

    for qaoa_type_name in qaoa_type_names:

      if qaoa_type_name == "Vanilla":

        cost_function = cost_function_qaoa_estimator
        cost_h = cost_hamiltonian
        qaoa = QAOA(p, num_qubits, cost_hamiltonian)
        circuit = qaoa.qc
        circuit.add_register(ClassicalRegister(num_qubits))

      elif qaoa_type_name == "Compact":
        cost_h = cost_hamiltonian_r
        cost_function = cost_function_qaoa_estimator
        qaoa_c = QAOA(p, num_qubits_c, cost_hamiltonian_r)
        circuit = qaoa_c.qc
        circuit.add_register(ClassicalRegister(num_qubits_c))

      elif qaoa_type_name == "QUMDA":
        cost_h = cost_hamiltonian
        cost_function = cost_function_qumda
        qumda = QUMDA(p,num_qubits, cost_hamiltonian, inital_pv)
        circuit = qumda.qc
        circuit.add_register(ClassicalRegister(num_qubits))

      else:
        cost_h = cost_hamiltonian
        cost_function = cost_function_qumda_w
        qumda = QUMDA(p,num_qubits, cost_hamiltonian, inital_pv)
        circuit = qumda.qc
        circuit.add_register(ClassicalRegister(num_qubits))


      circuit.measure_all()
      circuit_transpile = pm.run(circuit)
      observable_transpile = cost_h.apply_layout(layout=circuit_transpile.layout)


      for _ in range(repeat):

        pv = inital_pv
        pv_p = pv

        f_vals_list = []
        f_vals_opt = []

        objective_func_vals = []
        objective_func_vals_opt = []

        counts = []
        shots_list = []

        start = time.perf_counter()

        with Session(backend=chosen_backend) as session:
          # If using qiskit-ibm-runtime<0.24.0, change `mode=` to `session=`
          estimator = Estimator(mode=session)
          sampler = Sampler(mode=session)

          estimator.options.default_shots = shots
          sampler.options.default_shots = shots

          # Set simple error suppression/mitigation options
          #estimator.options.dynamical_decoupling.enable = True
          #estimator.options.dynamical_decoupling.sequence_type = "XY4"
          #estimator.options.twirling.enable_gates = True
          #estimator.options.twirling.num_randomizations = "auto"

          result = minimize(
              cost_function,
              init_params,
              args=(circuit_transpile, observable_transpile, estimator),
              method= "COBYLA",
          )
          #print(result)
          result_x = result.x

        #Optimal circuit
        if qaoa_type_name == "QUMDA" or qaoa_type_name == "QUMDA-W":
          pv_opt = pv if objective_func_vals[-1] > objective_func_vals[-2] else pv_p
          circuit_transpile_opt = qumda.update_distribution_qc(circuit_transpile, pv_opt)

        else:
          circuit_transpile_opt = circuit_transpile

        optimized_circuit_max_cut = circuit_transpile_opt.assign_parameters(result_x)

        #Sample of circuit
        sampler = Sampler(mode=chosen_backend)
        sampler.options.default_shots = shots
        pub= (optimized_circuit_max_cut, )
        #job = sampler.run([pub], shots=int(1e4))
        job = sampler.run([pub])

        counts_int = job.result()[0].data.meas.get_int_counts()

        shots = sum(counts_int.values())
        final_distribution = {key: val/shots for key, val in counts_int.items()}

        #Best solution
        best_solution = best_sample(final_distribution, cost_hamiltonian)

        best_solution_string = to_bitstring(int(best_solution), len(graph_sample))

        best_solution_string.reverse()


        #Cut value
        cut_value= evaluate_sample(best_solution_string, graph_sample)

        end = time.perf_counter()
        elapsed = end - start

        # Build file path
        if hardware:
            base_file_name = qaoa_type_name + ': ' + str(shots_amount[0]) + ", "+ str(p) + " R"
        else:
            base_file_name = qaoa_type_name + ': ' + str(shots_amount[0]) + ", "+ str(p) + " S"
        file_name = base_file_name
        file_path = os.path.join(save_dir, file_name + ".json")

        # Prepare the results for a specific graph
        obj_func_cost = [float(i) for i in objective_func_vals]
        f_vals_list = [float(i) for i in f_vals_list]
        results_x = [float(i) for i in result_x]

        results = {
            "cut_value": cut_value,
            "objective_func_cost": obj_func_cost,
            "objective_func_vals_opt": objective_func_vals_opt,
            "elapsed_time_seconds": round(elapsed, 4),
            "optimal_parameters": results_x,
            "final_distribution": final_distribution,
            "sample_distribution": counts,
            "o_func_opt_list": f_vals_opt,
            "shots_list": shots_list
        }

        # Load existing JSON if it exists
        if os.path.exists(file_path):
            with open(file_path, "r") as f:
                data = json.load(f)

        else:
            data = {}

        # Store or update metadata
        data["file_name"] = file_name

        # Initialize structure if needed
        if "graphs" not in data:
            data["graphs"] = {}

        # Generate unique graph name if it already exists
        existing_graphs = data["graphs"]
        base_name = graph_name
        counter = 0
        unique_name = base_name

        while unique_name in existing_graphs:
            unique_name = f"{base_name}_{counter}"
            counter += 1


        # Add or update this graph's results
        data["graphs"][unique_name] = results

        ordered_data = {
            "file_name": file_name,
            "graphs": data.get("graphs", {})
        }

        # Save updated JSON
        with open(file_path, "w") as f:
            json.dump(ordered_data, f, indent=4)

        print(f"Saved results for {graph_name} into {file_name}.json")


Saved results for 10_graph_25_complete into QUMDA-W: 1024, 3 R.json
